# MoS2 Flake Detection - Edge & Intensity Based

## New Approach:
Since color-based detection isn't working well, this notebook uses:
- **Intensity differences** between flakes and background
- **Edge detection** to find flake boundaries
- **Morphological analysis** for shape filtering
- **Watershed segmentation** for overlapping regions

Your flakes appear **darker** than the background, so we'll focus on that contrast.

In [1]:
# Install required packages
!pip install opencv-python-headless matplotlib numpy scipy scikit-image

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
from scipy import ndimage
from skimage import measure, morphology, filters, segmentation, feature
from skimage.segmentation import watershed
# Removed problematic import: from skimage.feature import peak_local_maxima
# Removed unused import: from scipy.spatial.distance import cdist
import os

# Create directories
os.makedirs('/content/images', exist_ok=True)
os.makedirs('/content/results', exist_ok=True)
os.makedirs('/content/debug', exist_ok=True)

print("✓ Environment setup complete")

zsh:1: command not found: pip


ModuleNotFoundError: No module named 'cv2'

In [ ]:
class EdgeBasedMoS2Analyzer:
    def __init__(self):
        self.debug_mode = True
        
    def analyze_intensity_profile(self, image_path):
        """Analyze intensity characteristics of the image"""
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
        
        # Plot intensity histogram
        plt.figure(figsize=(12, 8))
        
        plt.subplot(2, 3, 1)
        plt.imshow(img_rgb)
        plt.title('Original Image')
        plt.axis('off')
        
        plt.subplot(2, 3, 2)
        plt.imshow(gray, cmap='gray')
        plt.title('Grayscale')
        plt.axis('off')
        
        plt.subplot(2, 3, 3)
        plt.hist(gray.flatten(), bins=50, alpha=0.7)
        plt.title('Intensity Histogram')
        plt.xlabel('Intensity')
        plt.ylabel('Frequency')
        
        # Show different intensity thresholds
        thresholds = [100, 120, 140, 160]
        for i, thresh in enumerate(thresholds):
            plt.subplot(2, 3, 4 + i)
            binary = gray < thresh
            plt.imshow(binary, cmap='gray')
            plt.title(f'Threshold < {thresh}')
            plt.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Print statistics
        print(f"Intensity stats:")
        print(f"  Min: {gray.min()}")
        print(f"  Max: {gray.max()}")
        print(f"  Mean: {gray.mean():.1f}")
        print(f"  Std: {gray.std():.1f}")
        
        return img_rgb, gray
    
    def detect_by_intensity_difference(self, img_rgb, gray):
        """Detect flakes based on intensity differences"""
        # Method 1: Simple thresholding (flakes are darker)
        # Use Otsu's method to find optimal threshold
        thresh_otsu, binary_otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        # Method 2: Adaptive thresholding
        binary_adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                               cv2.THRESH_BINARY_INV, 15, 5)
        
        # Method 3: Manual threshold based on histogram analysis
        # Flakes appear to be in the darker range
        manual_thresh = 140  # Adjust based on your image
        binary_manual = (gray < manual_thresh).astype(np.uint8) * 255
        
        return {
            'otsu': binary_otsu,
            'adaptive': binary_adaptive, 
            'manual': binary_manual,
            'otsu_thresh': thresh_otsu
        }
    
    def detect_by_edge_analysis(self, img_rgb, gray):
        """Enhanced edge-based detection"""
        # Preprocessing: slight blur to reduce noise
        blurred = cv2.GaussianBlur(gray, (3, 3), 0)
        
        # Multiple edge detection methods
        edges_canny1 = cv2.Canny(blurred, 20, 60)
        edges_canny2 = cv2.Canny(blurred, 40, 100)
        edges_canny3 = cv2.Canny(blurred, 60, 140)
        
        # Combine edge maps
        edges_combined = cv2.bitwise_or(cv2.bitwise_or(edges_canny1, edges_canny2), edges_canny3)
        
        # Close small gaps in edges
        kernel_close = np.ones((3,3), np.uint8)
        edges_closed = cv2.morphologyEx(edges_combined, cv2.MORPH_CLOSE, kernel_close)
        
        # Fill enclosed regions
        contours, _ = cv2.findContours(edges_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        filled_mask = np.zeros_like(edges_closed)
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if 100 < area < 10000:  # Filter by reasonable flake sizes
                cv2.fillPoly(filled_mask, [contour], 255)
        
        return {
            'canny_low': edges_canny1,
            'canny_mid': edges_canny2,
            'canny_high': edges_canny3,
            'combined': edges_combined,
            'filled': filled_mask
        }
    
    def apply_morphological_filtering(self, binary_mask):
        """Apply morphological operations to clean up the mask"""
        # Remove small noise
        kernel_open = np.ones((2,2), np.uint8)
        opened = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel_open)
        
        # Fill small holes
        kernel_close = np.ones((4,4), np.uint8)
        closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel_close)
        
        return closed
    
    def extract_flake_contours(self, binary_mask, img_shape):
        """Extract and filter flake contours"""
        contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        flakes = []
        img_area = img_shape[0] * img_shape[1]
        
        for contour in contours:
            area = cv2.contourArea(contour)
            
            # Filter by size
            if area < 100 or area > img_area * 0.1:  # Too small or too large
                continue
            
            # Calculate shape properties
            perimeter = cv2.arcLength(contour, True)
            if perimeter == 0:
                continue
                
            # Approximate contour
            epsilon = 0.02 * perimeter
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            # Calculate shape metrics
            hull = cv2.convexHull(contour)
            hull_area = cv2.contourArea(hull)
            solidity = area / hull_area if hull_area > 0 else 0
            
            # Aspect ratio
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = max(w, h) / min(w, h) if min(w, h) > 0 else 1
            
            # Circularity
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            
            # Filter by shape characteristics
            is_flake = (
                0.4 < solidity < 1.0 and  # Not too irregular
                aspect_ratio < 4.0 and    # Not too elongated
                circularity > 0.2 and     # Not too thin/linear
                3 <= len(approx) <= 8     # Reasonable number of vertices
            )
            
            if is_flake:
                flakes.append({
                    'contour': contour,
                    'approx': approx,
                    'area': area,
                    'perimeter': perimeter,
                    'solidity': solidity,
                    'aspect_ratio': aspect_ratio,
                    'circularity': circularity,
                    'vertices': len(approx)
                })
        
        return flakes
    
    def visualize_detection_steps(self, img_rgb, gray, intensity_results, edge_results, final_flakes):
        """Visualize all detection steps"""
        fig, axes = plt.subplots(3, 4, figsize=(20, 15))
        
        # Row 1: Original and intensity-based methods
        axes[0,0].imshow(img_rgb)
        axes[0,0].set_title('Original Image')
        axes[0,0].axis('off')
        
        axes[0,1].imshow(intensity_results['otsu'], cmap='gray')
        axes[0,1].set_title(f'Otsu Threshold ({intensity_results["otsu_thresh"]:.0f})')
        axes[0,1].axis('off')
        
        axes[0,2].imshow(intensity_results['adaptive'], cmap='gray')
        axes[0,2].set_title('Adaptive Threshold')
        axes[0,2].axis('off')
        
        axes[0,3].imshow(intensity_results['manual'], cmap='gray')
        axes[0,3].set_title('Manual Threshold (<140)')
        axes[0,3].axis('off')
        
        # Row 2: Edge-based methods
        axes[1,0].imshow(edge_results['canny_low'], cmap='gray')
        axes[1,0].set_title('Canny Edges (Low)')
        axes[1,0].axis('off')
        
        axes[1,1].imshow(edge_results['canny_mid'], cmap='gray')
        axes[1,1].set_title('Canny Edges (Mid)')
        axes[1,1].axis('off')
        
        axes[1,2].imshow(edge_results['canny_high'], cmap='gray')
        axes[1,2].set_title('Canny Edges (High)')
        axes[1,2].axis('off')
        
        axes[1,3].imshow(edge_results['filled'], cmap='gray')
        axes[1,3].set_title('Edge-based Filled')
        axes[1,3].axis('off')
        
        # Row 3: Combined results and final detection
        # Test different methods and show results
        methods = ['otsu', 'adaptive', 'manual']
        method_results = []
        
        for i, method in enumerate(methods):
            cleaned_mask = self.apply_morphological_filtering(intensity_results[method])
            method_flakes = self.extract_flake_contours(cleaned_mask, img_rgb.shape)
            method_results.append((cleaned_mask, method_flakes))
            
            # Visualize
            result_img = img_rgb.copy()
            for j, flake in enumerate(method_flakes):
                cv2.drawContours(result_img, [flake['contour']], -1, (255, 0, 0), 2)
                # Add flake number
                M = cv2.moments(flake['contour'])
                if M['m00'] != 0:
                    cx = int(M['m10']/M['m00'])
                    cy = int(M['m01']/M['m00'])
                    cv2.putText(result_img, str(j+1), (cx-5, cy+5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
            
            axes[2,i].imshow(result_img)
            axes[2,i].set_title(f'{method.title()}: {len(method_flakes)} flakes')
            axes[2,i].axis('off')
        
        # Final combined result
        final_img = img_rgb.copy()
        colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255)]
        
        for i, flake in enumerate(final_flakes):
            color = colors[i % len(colors)]
            cv2.drawContours(final_img, [flake['contour']], -1, color, 2)
            
            # Add detailed flake info
            M = cv2.moments(flake['contour'])
            if M['m00'] != 0:
                cx = int(M['m10']/M['m00'])
                cy = int(M['m01']/M['m00'])
                cv2.putText(final_img, f"{i+1}", (cx-10, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        axes[2,3].imshow(final_img)
        axes[2,3].set_title(f'Final Result: {len(final_flakes)} flakes')
        axes[2,3].axis('off')
        
        plt.tight_layout()
        return fig, method_results
    
    def process_image(self, image_path):
        """Complete processing pipeline"""
        print(f"Processing: {Path(image_path).name}")
        
        # Load and analyze image
        img_rgb, gray = self.analyze_intensity_profile(image_path)
        
        # Apply different detection methods
        intensity_results = self.detect_by_intensity_difference(img_rgb, gray)
        edge_results = self.detect_by_edge_analysis(img_rgb, gray)
        
        # Choose the best method (you can modify this logic)
        # For now, let's try the manual threshold method
        best_mask = self.apply_morphological_filtering(intensity_results['manual'])
        final_flakes = self.extract_flake_contours(best_mask, img_rgb.shape)
        
        # Visualize results
        fig, method_results = self.visualize_detection_steps(img_rgb, gray, intensity_results, edge_results, final_flakes)
        
        return final_flakes, fig, method_results

# Initialize analyzer
analyzer = EdgeBasedMoS2Analyzer()
print("✓ Edge-based MoS2 Analyzer initialized")

In [ ]:
# Process all images
image_dir = Path('/content/images')
image_extensions = ['.jpg', '.jpeg', '.png', '.tiff', '.bmp']

image_files = []
for ext in image_extensions:
    image_files.extend(image_dir.glob(f'*{ext}'))
    image_files.extend(image_dir.glob(f'*{ext.upper()}'))

print(f"Found {len(image_files)} images to process\n")

all_results = []

for image_path in image_files:
    try:
        flakes, fig, method_results = analyzer.process_image(str(image_path))
        
        # Save figure
        plt.figure(fig.number)
        plt.savefig(f'/content/debug/{image_path.stem}_edge_detection.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Print results summary
        print(f"\n=== RESULTS FOR {image_path.name} ===")
        print(f"Total flakes detected: {len(flakes)}")
        
        if flakes:
            areas = [f['area'] for f in flakes]
            solidities = [f['solidity'] for f in flakes]
            
            print(f"Area range: {min(areas):.0f} - {max(areas):.0f} pixels")
            print(f"Average solidity: {np.mean(solidities):.2f}")
            
            print("\nFlake details:")
            for i, flake in enumerate(flakes):
                print(f"  Flake {i+1}: Area={flake['area']:.0f}, Vertices={flake['vertices']}, "
                      f"Solidity={flake['solidity']:.2f}, AspectRatio={flake['aspect_ratio']:.2f}")
        
        # Print method comparison
        print("\nMethod comparison:")
        method_names = ['Otsu', 'Adaptive', 'Manual']
        for name, (mask, flakes_method) in zip(method_names, method_results):
            print(f"  {name} method: {len(flakes_method)} flakes")
        
        # Save detailed results
        flake_data = []
        for i, flake in enumerate(flakes):
            flake_info = {
                'id': i + 1,
                'area': float(flake['area']),
                'perimeter': float(flake['perimeter']),
                'vertices': int(flake['vertices']),
                'solidity': float(flake['solidity']),
                'aspect_ratio': float(flake['aspect_ratio']),
                'circularity': float(flake['circularity'])
            }
            flake_data.append(flake_info)
        
        results = {
            'filename': image_path.name,
            'total_flakes': len(flakes),
            'detection_method': 'edge_intensity_based',
            'flake_details': flake_data
        }
        
        all_results.append(results)
        
        with open(f'/content/results/{image_path.stem}_edge_results.json', 'w') as f:
            json.dump(results, f, indent=2)
            
        print(f"\n✓ Results saved to /content/results/{image_path.stem}_edge_results.json")
        
    except Exception as e:
        print(f"❌ Error processing {image_path.name}: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

# Save summary
if all_results:
    with open('/content/results/edge_detection_summary.json', 'w') as f:
        json.dump(all_results, f, indent=2)
    
    print(f"\n=== FINAL SUMMARY ===")
    total_flakes = sum(r['total_flakes'] for r in all_results)
    print(f"Total images processed: {len(all_results)}")
    print(f"Total flakes detected: {total_flakes}")
    print(f"Average flakes per image: {total_flakes/len(all_results):.1f}")
else:
    print("No images were successfully processed.")

In [ ]:
# Manual threshold tuning - run this to find optimal threshold
def test_threshold_range(image_path, threshold_range):
    """Test a range of thresholds to find optimal value"""
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    
    for i, thresh in enumerate(threshold_range):
        row = i // 4
        col = i % 4
        
        # Apply threshold
        binary = (gray < thresh).astype(np.uint8) * 255
        
        # Clean up
        kernel = np.ones((3,3), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
        
        # Count objects
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        valid_contours = [c for c in contours if 100 < cv2.contourArea(c) < 10000]
        
        # Visualize
        result_img = img_rgb.copy()
        cv2.drawContours(result_img, valid_contours, -1, (255, 0, 0), 2)
        
        axes[row, col].imshow(result_img)
        axes[row, col].set_title(f'Threshold < {thresh}: {len(valid_contours)} objects')
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

# Test different thresholds
if image_files:
    print("Testing different threshold values...")
    test_thresholds = [120, 130, 140, 150, 160, 170, 180, 190]
    test_threshold_range(str(image_files[0]), test_thresholds)
else:
    print("No images available for threshold testing.")

## Analysis and Next Steps

### What This Notebook Does:
1. **Analyzes intensity characteristics** of your microscopy images
2. **Uses multiple detection approaches**:
   - Otsu's automatic thresholding
   - Adaptive thresholding
   - Manual threshold (flakes < background intensity)
   - Edge detection with gap filling
3. **Applies morphological filtering** to clean up results
4. **Filters by shape characteristics** (area, solidity, aspect ratio)

### Expected Improvements:
- Should detect **many more flakes** than previous versions
- Better **edge detection** for flake boundaries
- **Multiple validation methods** shown side-by-side

### If Results Need Further Tuning:
1. **Check the intensity histogram** to see optimal threshold range
2. **Adjust the manual threshold** in the code (currently 140)
3. **Modify size filters** (currently 100-10000 pixels)
4. **Change shape criteria** (solidity, aspect ratio, circularity)

### Once Stage 1 Works Well:
We can proceed to implement **Stage 2** (multilayer detection) and **Stage 3** (twist angle calculation) using the successfully detected flakes.
